# Coursera Capstone Project

### by Malte Jörg

This Jupyter Notebook will be mainly used for the IBM Applied Data Science Capstone Project! 

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
